# Important: do Cell > All Outputs > Clear before commiting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import qiskit
from qiskit import BasicAer, QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.aqua.circuits import FixedValueComparator

from qiskit.transpiler.passes import Unroller
from qiskit.converters import circuit_to_dag, dag_to_circuit
import re 
import time
from qiskit import IBMQ
import operator

In [2]:
NUM_BITS = 5
S =  list(range(10, 20))
N =  len(S)
simulator = BasicAer.get_backend('statevector_simulator')

In [3]:
myAPItoken = "PUT YOUR TOKEN HERE"
IBMQ.enable_account(myAPItoken)
IBMQ.stored_account()
print("Available backends:")
print(IBMQ.backends())

Available backends:


/home/julius/anaconda3/envs/qc/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:547: DeprecationWarning: IBMQ.backends() is being deprecated. Please use IBMQ.get_provider() to retrieve a provider and AccountProvider.backends() to find its backends.
  DeprecationWarning)


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmqx4') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-academic', group='u-tokyo', project='intro-qc-2019s')>, <IBMQBackend('ibmq_20_tokyo') from IBMQ(hub='ibm-q-academic', group='u-tokyo', project='intro-qc-2019s')>, <IBMQBackend('ibmq_poughkeepsie') from IBMQ(hub='ibm-q-academic', group='u-tokyo', project='intro-qc-2019s')>]


In [4]:
backend = IBMQ.get_backend('ibmq_20_tokyo')

/home/julius/anaconda3/envs/qc/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:594: DeprecationWarning: IBMQ.get_backend() is being deprecated. Please use IBMQ.get_provider() to retrieve a provider and AccountProvider.get_backend("name") to retrieve a backend.
  DeprecationWarning)


In [5]:
x = QuantumRegister(NUM_BITS, name="xs") #name cannot be x or the conversion to qasm string and back will FAIL!
work = QuantumRegister(NUM_BITS-1, name="a")
y = QuantumRegister(1, name="y")
x_meas = ClassicalRegister(NUM_BITS, name="m")


# y ^= (x == k)
def u_omega_eq(k):
    qc = QuantumCircuit(x, work, y)

    for i in range(NUM_BITS):
        if k & (1 << i):
            pass
        else:
            qc.x(x[i])

    qc.ccx(x[0], x[1], work[0])
    for i in range(2, NUM_BITS-1):
        qc.ccx(x[i], work[i-2], work[i-1])
    qc.ccx(x[-1], work[NUM_BITS-3], y)
    for i in range(NUM_BITS-2, 1, -1):
        qc.ccx(x[i], work[i-2], work[i-1])
    qc.ccx(x[0], x[1], work[0])
    

    for i in range(NUM_BITS):
        if k & (1 << i):
            pass
        else:
            qc.x(x[i])
    return qc

# y ^= (x < k)
def u_omega_cmp(k):
    qc = QuantumCircuit(x, work, y)
    cmp = FixedValueComparator(NUM_BITS, value=k, geq=False)
    cmp.build(qc, [x[i] for i in range(NUM_BITS)] + [y[0]], work)
    return qc

def a():
    #qc = QuantumCircuit(x)
    #qc.h(x[0])
    #qc.h(x[2])
    #return qc

    a = [0]*(2**NUM_BITS)
    for s in S:
        if s > (2**NUM_BITS):
            pass
        a[s] = 1
    a = [float(i) /np.sqrt(sum(a))  for i in a]
    
    
    
    qc = QuantumCircuit(x) 
    qc.initialize(a,[x[i] for i in range(NUM_BITS)]) 
    dag = circuit_to_dag(qc)
    pass_ = Unroller(basis=['u3', 'cx', 'id'])
    unrolled_dag = pass_.run(dag)
    unrolled_circuit = dag_to_circuit(unrolled_dag)
    import re
    qc_without_reset = re.sub("reset.*\n", "", unrolled_circuit.qasm())
      

   # print ( qc_without_reset) 
    qc = QuantumCircuit.from_qasm_str(qc_without_reset)
    
    return qc 

def a_inv():
    return a().inverse() 

def diffusion():
    qc = QuantumCircuit(x)
    qc += a_inv()
    qc += u_omega_eq(0)
    qc += a()
    return qc

def grover_iteration(u_omega):
    qc = QuantumCircuit()
    qc += u_omega
    qc.barrier()
    qc += diffusion()
    return qc


In [6]:
# TODO: wrong for comparison
THETA = 2*np.arcsin(1/np.sqrt(N))
ITERS = (np.pi/2 - THETA/2) / THETA

print(ITERS)
ITERS = int(np.round(ITERS))
print(ITERS)


num_grover_iters = {0, ITERS}
k = 1
while k <= ITERS:
    num_grover_iters.add(k)
    k *= 2   
#num_grover_iters = {0}
print(num_grover_iters)

def find_k(k):
    shots = 1
    
    qc = QuantumCircuit(x,work,y,x_meas)
    qc += a()
    qc.x(y)
    qc.h(y)
    for _ in range(ITERS):
        qc += grover_iteration(u_omega_eq(k))
    qc.measure(x, x_meas)
    result = execute(qc, simulator, shots=shots, seed_simulator=int(np.uint32(100*time.time()))).result()
    #print_statevector(result.get_statevector())
    counts = result.get_counts()
    k_bin = ("{:0%db}"%NUM_BITS).format(k)
    return k_bin in counts

def find_min(iters):
    working_min = 1<<NUM_BITS
    
    for _ in range(iters):
        experiments = []
        for k in num_grover_iters:
            qc = QuantumCircuit(x,work,y,x_meas)
            qc += a()
            qc.x(y)
            qc.h(y)
            for _ in range(k):
                qc += grover_iteration(u_omega_cmp(working_min))
            qc.measure(x, x_meas)
            experiments.append(qc)

        result = execute(experiments, simulator, shots=1, seed_simulator=int(np.uint32(100*time.time()))).result()
        for i in range(len(experiments)):
            counts = result.get_counts(i)
            print(counts)
        working_min = min(working_min, min([int(k, 2) for k in counts if counts[k] > 0]))
        print(working_min)
    return working_min



1.9410157268268091
2
{0, 1, 2}


In [7]:
def print_statevector(statevector):
    for i in range(len(statevector)):
        x_val = i & ((1<<NUM_BITS)-1)
        y_val = i >> (NUM_BITS + NUM_BITS-1)
        if np.abs(statevector[i]) > 1e-8:
            print("{:.3f} |{}>|{}>".format(statevector[i], x_val, y_val))

In [ ]:
# paper talks of 22.5 iterations
print('min', S, "=", find_min(5))

{'10001': 1}
{'10001': 1}
{'10001': 1}
17


In [ ]:
# this works most of the time
query =  13
print(query, "in", S, ":", find_k(query))

In [17]:
def find_k_real(k,old_run_ID = None):
    shots = 100
    qc = QuantumCircuit(x,work,y,x_meas)
    qc += a()
    qc.x(y)
    qc.h(y)
    for _ in range(ITERS):
        qc += grover_iteration(u_omega_eq(k))
    qc.measure(x, x_meas)
    if old_run_ID == None:
        
        result = execute(qc, backend, shots=shots, seed_simulator=int(np.uint32(100*time.time()))).result()
    else: 
        result =  backend.retrieve_job(old_run_ID).result() 
    #print_statevector(result.get_statevector())
    counts = result.get_counts()
    k_bin = ("{:0%db}"%NUM_BITS).format(k)
    return k_bin is max(counts.items(), key=operator.itemgetter(1))[0], counts

def find_min_real(iters,jobs_run):
    working_min = 1<<NUM_BITS
    
    for i in range(iters):
        experiments = []
        for k in num_grover_iters:
            qc = QuantumCircuit(x,work,y,x_meas)
            qc += a()
            qc.x(y)
            qc.h(y)
            for _ in range(k):
                qc += grover_iteration(u_omega_cmp(working_min))
            qc.measure(x, x_meas)
            experiments.append(qc)
        if (i < len(jobs_run)):
            
        else:
            result = execute(experiments, backend, shots=1, seed_simulator=int(np.uint32(100*time.time()))).result()
        for i in range(len(experiments)):
            counts = result.get_counts(i)
            print(counts)
        working_min = min(working_min, min([int(k, 2) for k in counts if counts[k] > 0]))
        print(working_min)
    return working_min



In [15]:
query = 11
t,c = find_k_real(query,"5d432830523a2e0012628a76")
t

False

In [ ]:
# soooo this worked twice out of 5 tries before the presentation and then none out of 20 afterwards, 
#upping the shot count will make the code run basically indefinitely(two runs took almost 2 days, before i cancelled) 
#if given more time (after the project is done, we might be able find a solution) 
print('min', S, "=", find_min_real(5))

{'01011': 1}
{'00000': 1}
{'01111': 1}
15
{'10110': 1}
{'10110': 1}
{'00101': 1}
5
{'00110': 1}
{'11010': 1}
{'10101': 1}
5


In [ ]:
old_find_jobs = {0: 5d43924d3b332300120b9959}